<a href="https://colab.research.google.com/github/greyhound101/hardhat/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os

In [ ]:
df=pd.read_csv('../input/notebookd5efbec70e/predicted.csv')
df[['XMin','XMax','YMin','YMax']]=df[['XMin','XMax','YMin','YMax']].astype('int')
df.shape

In [ ]:
from tqdm import tqdm
import cv2
import gc
h=[]
w=[]
for i in tqdm(df['ImageID'].values):
    img=cv2.imread('../input/hasrhat-test/Images/hard_hat_workers'+str(i)+'.png')
    h.append(img.shape[0])
    w.append(img.shape[1])
    del([img])
    gc.collect()


In [ ]:
# df=df.drop(['LabelName'],1)
df['heights']=h
df['width']=w
df['depth']=3
df.head()

/content/yolov5


In [ ]:
df['ImageID'].nunique()

In [ ]:
import xml.etree.ElementTree as ET
for id in df['ImageID'].unique():
    tmp=df.loc[df['ImageID']==id]
    i=tmp.index[0]
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = 'images'
    ET.SubElement(annotation, 'filename').text = str(df['ImageID'].iloc[i])
    ET.SubElement(annotation, 'path').text = '/home/elliot/Documents/mmdetection_object_detection_demo/data/vedai/train_images/'
    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'
    ET.SubElement(annotation, 'segmented').text = '0'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(df['heights'].iloc[i])
    ET.SubElement(size, 'height').text = str(df['width'].iloc[i])
    ET.SubElement(size, 'depth').text = str(3)
    for idx in tmp.index:
        ob = ET.SubElement(annotation, 'object')
        ET.SubElement(ob, 'name').text = 'hardhat'
        ET.SubElement(ob, 'pose').text = 'Unspecified'
        ET.SubElement(ob, 'truncated').text = '0'
        ET.SubElement(ob, 'difficult').text = '0'
        bbox = ET.SubElement(ob, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = str(df['XMin'].iloc[idx])
        ET.SubElement(bbox, 'ymin').text = str(df['YMin'].iloc[idx])
        ET.SubElement(bbox, 'xmax').text = str(df['XMax'].iloc[idx])
        ET.SubElement(bbox, 'ymax').text = str(df['YMax'].iloc[idx])

        fileName = str(df['ImageID'].iloc[idx])
    tree = ET.ElementTree(annotation)
    tree.write(fileName + ".xml", encoding='utf8')
